In [ ]:
import logging
import os
import requests
import pandas as pd
import duckdb

logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [ ]:
STORE_ENDPOINT = "https://fakestoreapi.com/products/category/women%27s%20clothing"

r = requests.get(STORE_ENDPOINT).json()
store_items = pd.DataFrame(r)

#Explode "rating" (dict to columns)
store_items = store_items.join(pd.DataFrame(store_items["rating"].tolist())[["rate", "count"]])
store_items.drop("rating", axis=1, inplace=True)

store_items.rename(columns={"price": "price_usd", "rate": "rating_rate", "count": "rating_count"}, inplace=True)

store_items.head()

In [ ]:
def get_file_paths(directory):
    # get full path and basename for all files in directory
    files = []
    for entry in os.scandir(f'{directory}/'):
        if entry.is_file():
            files.append({"path": entry.path, "name": os.path.basename(entry.path)})
        else:
            # if the entry is a directory, recursively call get_file_paths
            files.extend(get_file_paths(entry.path))
    return files

In [ ]:
# Get ecommerce data
dates = ['2024-02-12', '2024-02-13']

data_path = '../data'
rates_path = f'{data_path}/exchange_rates'

rate_files = get_file_paths(rates_path)

# I'm pulling all the rates datasets, but in a real scenario I would only pull the required dates and currencies (for example, using AWS S3 and 
# the AWS Athena query engine over parquet files)
rates_df = pd.concat((pd.read_csv(f["path"]) for f in rate_files), ignore_index=True)

# Only keep USD rates for the required dates
rates_df = rates_df[(rates_df["date"].isin(dates)) & (rates_df["from_currency"] == "USD")]
# Drop unneeded columns
rates_df.drop(columns=['from_currency', 'amount', 'to_currency','updated_at'], inplace=True)
rates_df.rename(columns={'date': 'exchange_rate_date'}, inplace=True)

rates_df.head()

In [ ]:
products_df = store_items.merge(rates_df, how='cross')
products_df["price_eur"] = products_df["price_usd"] * products_df["exchange_rate"]
print(products_df.columns)
products_df.head()

In [ ]:
# I'm using the DuckDB library to mimic a relational database
# Check the "DDL_products_big_query.sql" file for the actual table creation script for BigQuery

# Create statement for the final table
ddl_query = """
        -- Drop the table if it exists
        DROP TABLE IF EXISTS top_products;

        -- Create the table with appropriate data types
        CREATE TABLE top_products (
            id INTEGER,
            title STRING,
            price_usd FLOAT,
            description STRING,
            category STRING,
            image STRING,
            rating_rate FLOAT,
            rating_count INTEGER,
            exchange_rate_date DATE,
            exchange_rate FLOAT,
            price_eur FLOAT
        );
    """
# I'm assuming that the required Top 5 products should be just five items, otherwise this would be a DENSE_RANK().
# In a real scenario this would be improved, for example, using other fields for cases where there are multiple items with the exact same rank.
top_products_query ="""
        INSERT INTO
            top_products
        SELECT
            id,
            title,
            price_usd,
            description,
            category,
            image,
            rating_rate,
            rating_count,
            exchange_rate_date,
            exchange_rate,
            price_eur
        FROM
            (
                SELECT
                    id,
                    title,
                    price_usd,
                    description,
                    category,
                    image,
                    rating_rate,
                    rating_count,
                    exchange_rate_date,
                    exchange_rate,
                    price_eur,
                    rank() OVER (
                        PARTITION BY exchange_rate_date
                        ORDER BY
                            rating_rate DESC
                    ) AS rank
                FROM
                    all_products
            )
        WHERE
            rank <= 5
    """

# Get the data from the "products_df" DataFrame into DuckDB
duckdb.sql("CREATE OR REPLACE TABLE all_products AS SELECT * FROM products_df")
# Create the "top_products" table
duckdb.sql(ddl_query)

In [ ]:

# Insert the top products into the "top_products" table using the "top_products_query" query
duckdb.sql(top_products_query)